In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-05-03 10:53:08.193942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Expanded dataset
input_texts = [
    'hi', 'hello', 'how are you', 'bye', 'what is your name', 'thanks',
    'what do you do', 'who made you', 'how old are you', 'what is ai',
    'tell me a joke', 'what is the weather', 'good morning', 'good night',
    'are you real', 'i love you', 'can you help me', 'what is your job',
    'where are you from', 'do you like humans',
]

target_texts = [
    'hello', 'hi', 'i am fine, thank you', 'goodbye', 'i am a bot',
    'you are welcome', 'i help users with questions', 'i was made by OpenAI',
    'i do not age', 'ai means artificial intelligence',
    'why did the computer get cold? because it left its windows open!',
    'i cannot check the weather, but i hope it’s sunny!',
    'good morning!', 'good night!', 'not really, just code',
    'i love chatting with you', 'yes, how can i assist you?',
    'my job is to talk with you', 'i live on the internet',
    'i think humans are amazing!',
]

In [6]:
# Add <start> and <end> tokens to target sentences
target_texts = ['<start> ' + text + ' <end>' for text in target_texts]

In [26]:
# Tokenize input and target texts
input_tokenizer = Tokenizer()
target_tokenizer = Tokenizer(filters='')  # Keeps special tokens like <start> and <end>

input_tokenizer.fit_on_texts(input_texts)
target_tokenizer.fit_on_texts(target_texts)

input_sequences = input_tokenizer.texts_to_sequences(input_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)

max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in target_sequences)

encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences([seq[:-1] for seq in target_sequences], maxlen=max_decoder_seq_length-1, padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in target_sequences], maxlen=max_decoder_seq_length-1, padding='post')


In [10]:
# Vocabulary sizes
num_encoder_tokens = len(input_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

In [12]:
# One-hot encode decoder target data
decoder_target_data_oh = tf.keras.utils.to_categorical(decoder_target_data, num_classes=num_decoder_tokens)


In [14]:
# Build Seq2Seq model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=256)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data_oh, batch_size=2, epochs=300, verbose=0)

In [16]:
# Inference models
# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding2 = Embedding(input_dim=num_decoder_tokens, output_dim=256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_embedding2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [30]:
# Reverse-lookup for words
reverse_target_index = {i: word for word, i in target_tokenizer.word_index.items()}

def decode_sequence(input_seq):
    input_seq = input_tokenizer.texts_to_sequences([input_seq])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_seq_length, padding='post')
    states_value = encoder_model.predict(input_seq)

    target_seq = np.array([[target_tokenizer.word_index['<start>']]])
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_index.get(sampled_token_index, '')
    
        if sampled_word == '<end>' or sampled_word == '':
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
    
            target_seq = np.array([[sampled_token_index]])
            states_value = [h, c]


    return decoded_sentence.strip()

In [ ]:
# Chat loop
print("Chatbot is ready! Type 'exit' to stop.")
while True:
    user_input = input("You: ").lower()
    if user_input == 'exit':
        print("Bot: Goodbye!")
        break
    response = decode_sequence(user_input)
    print("Bot:", response)

Chatbot is ready! Type 'exit' to stop.


You:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bot: hello


You:  how are you 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: i am fine, thank


You:  what is your name 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: i am a bot
